In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [22]:
import numpy as np
import pandas as pd

df = pd.read_csv('./kaggle/input/spaceship-titanic/train_VIPNull_update.csv')
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,1.0,0.0,1.0,0.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,0.0,0.0,5.0,0.0,1.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,1.0,0.0,0.0,0.0,1.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,1.0,0.0,0.0,0.0,1.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,0.0,0.0,5.0,1.0,1.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,2747_01,2.0,1.0,5.0,570.0,0.0,2.0,49.0,0.0,0.0,0.0,0.0,0.0,0.0,Stisps Chrie,1.0
8689,2822_02,0.0,0.0,6.0,450.0,1.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,NaN,Salley Harverez,0.0
8690,5777_01,0.0,0.0,5.0,1199.0,0.0,1.0,46.0,0.0,559.0,25.0,NaN,22.0,765.0,Katen River,0.0
8691,6035_02,0.0,1.0,6.0,976.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,Ginald Morsentley,0.0


컬럼
- Destination 목적지 {55 Cancri e, PSO J318.5-22, TRAPPIST-1e}
- HomePlanet  {Earth , Europa, Mars}
- Cabin의 종류 6560
    - Cabin을 앞 대문자만 남기면 의미가 있을 수 있을까?
    - 돈이 많은 사람이 사는 자리 등으로 의미가 있는지확인
    - A,B,C,D,E,F,G,T로 변환 후 다시 숫자로 0~7까지 변환
- True / False 
    - CryoSleep
    - VIP
    - Transported(target)
- 삭제
    - PassengerId
    - Name

### null값 처리
XGBOOST를 사용해서 pridct값을 null값으로 대체

In [23]:
# df_destination =  df_train중에서 Destination의 null값이 존재하는 열만 추출 
# -> 훈련된 모델의 값으로 다시 채울 예정
df_destination = df[df['Destination'].isnull()]

#df_train 중에서 Destination의 null값이 존재하는 열만 추출 -> 추후 훈련할 데이터프레임
df_train01 = df.dropna(subset=['Destination'])
# df_train
df[df['Destination'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
46,0045_02,2.0,1.0,5.0,10.0,0.0,NaN,19.0,0.0,0.0,0.0,0.0,0.0,0.0,Mass Chmad,1.0
120,0138_02,0.0,0.0,4.0,5.0,0.0,NaN,34.0,0.0,0.0,22.0,0.0,564.0,207.0,Monah Gambs,0.0
131,0152_01,0.0,0.0,5.0,32.0,0.0,NaN,41.0,0.0,0.0,0.0,0.0,0.0,607.0,Andan Estron,0.0
403,0462_01,0.0,1.0,6.0,67.0,1.0,NaN,50.0,0.0,0.0,0.0,0.0,0.0,0.0,Ronia Sosanturney,0.0
439,0504_03,1.0,1.0,1.0,19.0,1.0,NaN,24.0,0.0,0.0,0.0,0.0,0.0,0.0,Sterops Unpasine,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8473,8574_02,2.0,0.0,3.0,258.0,1.0,NaN,53.0,0.0,1042.0,0.0,2.0,1.0,11.0,Yakers Welte,0.0
8575,4351_01,0.0,0.0,5.0,895.0,0.0,NaN,21.0,0.0,641.0,0.0,409.0,0.0,0.0,Timmy Carezquez,0.0
8586,5070_01,0.0,0.0,5.0,966.0,1.0,NaN,20.0,0.0,542.0,175.0,0.0,5.0,0.0,Lillia Collinson,1.0
8661,8602_03,0.0,1.0,6.0,1396.0,0.0,NaN,24.0,0.0,0.0,0.0,0.0,0.0,0.0,Elson Weissey,0.0


In [24]:
# train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train01.drop(['PassengerId','Destination','Name'],axis=1),df_train01['Destination'], test_size=0.2, random_state=42)

# scaler = StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
X_train.shape
y_train.shape
X_test.shape
y_test.shape

(6808, 13)

(6808,)

(1703, 13)

(1703,)

In [25]:
#XGBoost1
import xgboost as xgb
import pandas as pd
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = xgb.XGBClassifier(early_stopping_rounds=20, reg_alpha = 0.2)
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)#, verbose=True

pred = model.predict(X_test)
acc = accuracy_score(y_test,pred)
acc

# from sklearn.metrics import confusion_matrix
# import scikitplot as skplt

# pred = model.predict(X_test)
# # 실제 , 예측
# cm = confusion_matrix(y_test, pred)
# print('confusion_matrix : \n', cm)

# skplt.metrics.plot_confusion_matrix(y_test,pred,figsize=(8,6))
# plt.show()

# cl_report = metrics.classification_report(y_test,pred)
# print('리포트:\n',cl_report)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=20, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

0.7146212566059894

In [26]:
# df_01 은 df_train 중에서 Destination의 값이 null인 경우의 데이터프레임
df_destination_null = df_destination.drop('Destination',axis=1)
df_destination_null

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
46,0045_02,2.0,1.0,5.0,10.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,Mass Chmad,1.0
120,0138_02,0.0,0.0,4.0,5.0,0.0,34.0,0.0,0.0,22.0,0.0,564.0,207.0,Monah Gambs,0.0
131,0152_01,0.0,0.0,5.0,32.0,0.0,41.0,0.0,0.0,0.0,0.0,0.0,607.0,Andan Estron,0.0
403,0462_01,0.0,1.0,6.0,67.0,1.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,Ronia Sosanturney,0.0
439,0504_03,1.0,1.0,1.0,19.0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,Sterops Unpasine,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8473,8574_02,2.0,0.0,3.0,258.0,1.0,53.0,0.0,1042.0,0.0,2.0,1.0,11.0,Yakers Welte,0.0
8575,4351_01,0.0,0.0,5.0,895.0,0.0,21.0,0.0,641.0,0.0,409.0,0.0,0.0,Timmy Carezquez,0.0
8586,5070_01,0.0,0.0,5.0,966.0,1.0,20.0,0.0,542.0,175.0,0.0,5.0,0.0,Lillia Collinson,1.0
8661,8602_03,0.0,1.0,6.0,1396.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,Elson Weissey,0.0


In [27]:
# 훈련된 xgboost모델을 사용해서 df_destination_null의 HomePlanet값을 예측
pred = model.predict(df_destination_null.drop(['PassengerId','Name'],axis=1))
pred

array([2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 0, 2, 0, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0,
       0, 2, 2, 2, 2, 2], dtype=int64)

In [28]:

# 예측된 값을 다시 df_destination으로 삽입(null값 채우기)
df_destination['Destination'] = pred
df_destination

C:\Users\shs03\AppData\Local\Temp\ipykernel_62500\4030680946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_destination['Destination'] = pred


,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
46,0045_02,2.0,1.0,5.0,10.0,0.0,2,19.0,0.0,0.0,0.0,0.0,0.0,0.0,Mass Chmad,1.0
120,0138_02,0.0,0.0,4.0,5.0,0.0,2,34.0,0.0,0.0,22.0,0.0,564.0,207.0,Monah Gambs,0.0
131,0152_01,0.0,0.0,5.0,32.0,0.0,2,41.0,0.0,0.0,0.0,0.0,0.0,607.0,Andan Estron,0.0
403,0462_01,0.0,1.0,6.0,67.0,1.0,2,50.0,0.0,0.0,0.0,0.0,0.0,0.0,Ronia Sosanturney,0.0
439,0504_03,1.0,1.0,1.0,19.0,1.0,0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,Sterops Unpasine,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8473,8574_02,2.0,0.0,3.0,258.0,1.0,2,53.0,0.0,1042.0,0.0,2.0,1.0,11.0,Yakers Welte,0.0
8575,4351_01,0.0,0.0,5.0,895.0,0.0,2,21.0,0.0,641.0,0.0,409.0,0.0,0.0,Timmy Carezquez,0.0
8586,5070_01,0.0,0.0,5.0,966.0,1.0,2,20.0,0.0,542.0,175.0,0.0,5.0,0.0,Lillia Collinson,1.0
8661,8602_03,0.0,1.0,6.0,1396.0,0.0,2,24.0,0.0,0.0,0.0,0.0,0.0,0.0,Elson Weissey,0.0


In [29]:

# null값을 채운 dataframe과 df_train01 합치기 
df_update = pd.concat([df_train01,df_destination],axis=0)
df_update

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,1.0,0.0,1.0,0.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,0.0,0.0,5.0,0.0,1.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,1.0,0.0,0.0,0.0,1.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,1.0,0.0,0.0,0.0,1.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,0.0,0.0,5.0,1.0,1.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8473,8574_02,2.0,0.0,3.0,258.0,1.0,2.0,53.0,0.0,1042.0,0.0,2.0,1.0,11.0,Yakers Welte,0.0
8575,4351_01,0.0,0.0,5.0,895.0,0.0,2.0,21.0,0.0,641.0,0.0,409.0,0.0,0.0,Timmy Carezquez,0.0
8586,5070_01,0.0,0.0,5.0,966.0,1.0,2.0,20.0,0.0,542.0,175.0,0.0,5.0,0.0,Lillia Collinson,1.0
8661,8602_03,0.0,1.0,6.0,1396.0,0.0,2.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,Elson Weissey,0.0


In [30]:
df_update[df_update['Destination'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported


In [31]:
df_update.to_csv('./kaggle/input/spaceship-titanic/train_DestinationNull_update.csv', index=False)
